In [8]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines
from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner
from transformers.trainer_callback import TrainerCallback
import itertools

In [9]:
model_name = "EleutherAI/pythia-70m"
# Get the current directory
current_directory = os.getcwd()
# Join the folder path
folder_path = os.path.join(current_directory, "content")
dataset_name = "ai-medical-chatbot_processed.jsonl"
dataset_path = os.path.join(folder_path, dataset_name)
#dataset_path = f"/content/{dataset_name}"
use_hf = False
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)
base_model = AutoModelForCausalLM.from_pretrained(model_name)
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

2024-04-09 23:29:45,708 - DEBUG - utilities - Config: datasets.path: c:\Blog\How-to-Finetuning-Large-Language-Models\content\ai-medical-chatbot_processed.jsonl
datasets.use_hf: false
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize False c:\Blog\How-to-Finetuning-Large-Language-Models\content\ai-medical-chatbot_processed.jsonl


2024-04-09 23:29:46,097 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-f1c6af33428df321/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/dataset_info.json
2024-04-09 23:29:46,128 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-f1c6af33428df321/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/dataset_info.json
2024-04-09 23:29:47,276 - DEBUG - utilities - Select CPU device


In [10]:
from collections import Counter
import math

def cosine_similarity(str1, str2):
    """
    Computes the cosine similarity between two strings using the Bag-of-Words model.

    Args:
        str1: The first string.
        str2: The second string.

    Returns:
        A float representing the cosine similarity between the two strings.
    """
    # Tokenize the strings
    tokens1 = str1.split()
    tokens2 = str2.split()

    # Create bag of words for each string
    bow1 = Counter(tokens1)
    bow2 = Counter(tokens2)

    # Get the set of all unique words
    all_words = set(bow1.keys()).union(set(bow2.keys()))

    # Compute dot product
    dot_product = sum(bow1[word] * bow2[word] for word in all_words)

    # Compute magnitudes
    magnitude1 = math.sqrt(sum(bow1[word] ** 2 for word in all_words))
    magnitude2 = math.sqrt(sum(bow2[word] ** 2 for word in all_words))

    # Compute cosine similarity
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    else:
        return dot_product / (magnitude1 * magnitude2)


In [20]:
def inference_new(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
      text,
      return_tensors="pt",
      truncation=True,
      max_length=max_input_tokens
  )

  # Generate
  device = model.device
  attention_mask = torch.ones_like(input_ids)  # Create mask with all 1s

  # Fix: Mask all padding tokens, including the first element
  attention_mask[input_ids == tokenizer.pad_token_id] = 0

  generated_tokens_with_prompt = model.generate(
      input_ids.to(device),
      max_length=max_output_tokens,
      attention_mask=attention_mask,
      pad_token_id=tokenizer.eos_token_id  # Set pad token
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]
  return generated_text_answer



In [33]:
from transformers import AutoModelForCausalLM
import shutil
def train_model(hyperparameters, delete=False):
  max_steps = hyperparameters["max_steps"]


  # Convert hyperparameter values to integers and add them to the string
  hyperparameter_str = '_'.join(str(int(value)) if isinstance(value, (int, float)) else value for value in hyperparameters.values())
  # Create the trained_model_name variable
  trained_model_name = f"ai_medical_{hyperparameter_str}"

  #trained_model_name = f"ai_medical_{max_steps}_steps"
  output_dir = trained_model_name
  training_args = TrainingArguments(
    # Learning rate
    learning_rate=hyperparameters["learning_rate"],

    # Number of training epochs
    num_train_epochs=hyperparameters["num_train_epochs"],

    # Max steps to train for (each step is a batch of data)
    # Overrides num_train_epochs, if not -1
    max_steps=max_steps,

    # Batch size for training
    per_device_train_batch_size=hyperparameters["per_device_train_batch_size"],

    # Directory to save model checkpoints
    output_dir=output_dir,

    # Other arguments
    overwrite_output_dir=False, # Overwrite the content of the output directory
    disable_tqdm=False, # Disable progress bars
    eval_steps=120, # Number of update steps between two evaluations
    save_steps=120, # After # steps model is saved
    warmup_steps=1, # Number of warmup steps for learning rate scheduler
    per_device_eval_batch_size=1, # Batch size for evaluation
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    optim=hyperparameters["optim"],
    gradient_accumulation_steps = hyperparameters['gradient_accumulation_steps'],
    gradient_checkpointing=False,
    # Parameters for early stopping
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
  )
  base_model.to(device)
  model_flops = (
    base_model.floating_point_ops(
      {
        "input_ids": torch.zeros(
            (1, training_config["model"]["max_length"])
        )
      }
    )
    * training_args.gradient_accumulation_steps
  )

  #print(base_model)
  print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
  print("Flops", model_flops / 1e9, "GFLOPs")

  trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
  training_output = trainer.train()
  # Evaluate the model
  eval_results = trainer.evaluate()

  # Adding Evaluation 
  save_dir = f'{output_dir}'
  trainer.save_model(save_dir)
  print("Saved model to:", save_dir)
  finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
  finetuned_slightly_model.to(device)
  test_question = test_dataset[0]['question']
  print("Question input (test):", test_question)
  predicted_answer=inference_new(test_question, finetuned_slightly_model, tokenizer)
  print("Finetuned slightly model's answer: ")
  print(predicted_answer) 
  test_answer = test_dataset[0]['answer']
  print("Target answer output (test):", test_answer)
  metric_cosine_similarity=cosine_similarity(test_answer, predicted_answer)
  print("Cosine Similarity:", metric_cosine_similarity)
  # Deleting the folder to save space
  if delete:
    shutil.rmtree(save_dir)
    print("Deleted model folder:", save_dir)
  return eval_results, training_output, metric_cosine_similarity


In [34]:
hyperparameters={'learning_rate': 1e-06,
'num_train_epochs': 1,
'per_device_train_batch_size': 1,
'optim': 'adafactor',
'num_iterations': 1,
'max_steps':3,
'gradient_accumulation_steps':2}

In [35]:
eval_results, training_output, metric_cosine_similarity =train_model(hyperparameters, delete=True)

Memory footprint 0.30687256 GB
Flops 1097.833906176 GFLOPs


  0%|          | 0/3 [00:00<?, ?it/s]

2024-04-10 00:00:08,175 - DEBUG - utilities - Step (1) Logs: {'loss': 4.4618, 'learning_rate': 1e-06, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}


{'loss': 4.4618, 'learning_rate': 1e-06, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}


2024-04-10 00:00:09,039 - DEBUG - utilities - Step (2) Logs: {'loss': 4.2251, 'learning_rate': 5e-07, 'epoch': 0.0, 'iter_time': 0.863516092300415, 'flops': 1271353152494.6572, 'remaining_time': 0.863516092300415}


...

2024-04-10 00:00:09,819 - DEBUG - utilities - Step (3) Logs: {'loss': 4.1862, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 0.8219277858734131, 'flops': 1335681704700.3687, 'remaining_time': 0.0}
2024-04-10 00:00:09,824 - DEBUG - utilities - Step (3) Logs: {'train_runtime': 2.5603, 'train_samples_per_second': 2.343, 'train_steps_per_second': 1.172, 'total_flos': 91932868608.0, 'train_loss': 4.290998458862305, 'epoch': 0.01, 'iter_time': 0.8244308233261108, 'flops': 1331626468970.2683, 'remaining_time': 0.0}


{'loss': 4.1862, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 0.8219277858734131, 'flops': 1335681704700.3687, 'remaining_time': 0.0}
{'train_runtime': 2.5603, 'train_samples_per_second': 2.343, 'train_steps_per_second': 1.172, 'train_loss': 4.290998458862305, 'epoch': 0.01, 'iter_time': 0.8244308233261108, 'flops': 1331626468970.2683, 'remaining_time': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

2024-04-10 00:00:16,872 - DEBUG - utilities - Step (3) Logs: {'eval_loss': 4.449901580810547, 'eval_runtime': 7.0419, 'eval_samples_per_second': 14.201, 'eval_steps_per_second': 14.201, 'epoch': 0.01, 'iter_time': 4.348652124404907, 'flops': 252453835066.4767, 'remaining_time': 0.0}


Saved model to: ai_medical_0_1_1_adafactor_1_3_2
Question input (test): Will Kalarchikai cure multiple ovarian cysts in PCOD?
Finetuned slightly model's answer: 


I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long list of cases. I have a very long l

In [5]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from ai-medical-chatbot: {test_dataset[0]['answer']}")
print("Model's answer: ")
#print(inference_new(test_text, base_model, tokenizer))

Question input (test): Will Kalarchikai cure multiple ovarian cysts in PCOD?
Correct answer from ai-medical-chatbot: Hello. I just read your query. See Kalarachi Kai choornam is helpful in amenorrhea. As far as small cysts are concerned they are unmatured eggs which failed to induce menstrual cycle previously, as a result, they got collected in the ovary and they will remain in the ovary. Now, you have got your periods you can start trying for conception. But I advise you to do it under the supervision of a nearby gynecologist because egg size is important while conception and that you can know by ovulation study. Ovulation study is performed under the supervision of a gynecologist. For gall stones, surgical intervention is required generally. Medicine is not of much help.
Model's answer: 


In [36]:
from tqdm import tqdm
import pandas as pd

In [37]:
def find_best_hyperparameters():
    best_hyperparameters = None
    best_loss = float('inf')
    # Lists to store data
    hyperparameters_list = []
    eval_results_list = []
    training_output_list = []
    cosine_similarity_list = []
    
    # Define hyperparameter search space
    hyperparameter_space = {
        "learning_rate": [1e-6, 1e-5, 1e-4],
        "num_train_epochs": [1,5,10,20],
        "per_device_train_batch_size": [1],
        "optim": ["adafactor"],
        "num_iterations": [1],
        "max_steps": [3],
        "gradient_accumulation_steps": [3],
    }
    # Generate all combinations of hyperparameters
    all_hyperparameters = list(itertools.product(*hyperparameter_space.values()))

    # Assuming all_hyperparameters is a list of hyperparameter combinations
    for hyperparameter_values in tqdm(all_hyperparameters):
        hyperparameters = dict(zip(hyperparameter_space.keys(), hyperparameter_values))
        
        # Evaluate the model
        # Print the current hyperparameters
        print("Using hyperparameters:")
        for key, value in hyperparameters.items():
            print(f"{key}: {value}")
        eval_results, training_output, metric_cosine_similarity = train_model(hyperparameters,delete=True)
        
        # Append data to lists
        hyperparameters_list.append(hyperparameters)
        eval_results_list.append(eval_results)
        training_output_list.append(training_output)
        cosine_similarity_list.append(metric_cosine_similarity)

        # Check if this set of hyperparameters gives better results
        if eval_results["eval_loss"] < best_loss:
                best_loss = eval_results["eval_loss"]
                best_hyperparameters = hyperparameters

    # Create DataFrame
    data = {
        'Hyperparameters': hyperparameters_list,
        'Evaluation Results': eval_results_list,
        'Training Output': training_output_list,
        'Cosine Similarity': cosine_similarity_list
    }
    df = pd.DataFrame(data)
    
    return best_hyperparameters, best_loss, df

In [38]:
# Call the function to find the best hyperparameters
best_hyperparameters, best_loss ,df = find_best_hyperparameters()

  0%|          | 0/12 [00:00<?, ?it/s]

Using hyperparameters:
learning_rate: 1e-06
num_train_epochs: 1
per_device_train_batch_size: 1
optim: adafactor
num_iterations: 1
max_steps: 3
gradient_accumulation_steps: 3
Memory footprint 0.30687256 GB
Flops 1646.750859264 GFLOPs


  0%|          | 0/3 [00:00<?, ?it/s]

2024-04-10 00:08:29,201 - DEBUG - utilities - Step (1) Logs: {'loss': 4.3719, 'learning_rate': 1e-06, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}
  0%|          | 0/12 [00:01<?, ?it/s]

{'loss': 4.3719, 'learning_rate': 1e-06, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}


...

  0%|          | 0/100 [00:00<?, ?it/s]

2024-04-10 00:13:59,853 - DEBUG - utilities - Step (3) Logs: {'eval_loss': 5.07048225402832, 'eval_runtime': 7.4669, 'eval_samples_per_second': 13.392, 'eval_steps_per_second': 13.392, 'epoch': 0.01, 'iter_time': 4.797343850135803, 'flops': 343263045282.31464, 'remaining_time': 0.0}


Saved model to: ai_medical_0_20_1_adafactor_1_3_3
Question input (test): Will Kalarchikai cure multiple ovarian cysts in PCOD?


100%|██████████| 12/12 [05:50<00:00, 29.18s/it]

Finetuned slightly model's answer: 
Hi. I hope this helps. I hope. For more information consult a urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online --> urologist online -->

In [39]:
print("Best hyperparameters:", best_hyperparameters)
print("Best loss:", best_loss)

Best hyperparameters: {'learning_rate': 1e-05, 'num_train_epochs': 20, 'per_device_train_batch_size': 1, 'optim': 'adafactor', 'num_iterations': 1, 'max_steps': 3, 'gradient_accumulation_steps': 3}
Best loss: 3.747753620147705


In [52]:
# Sort the DataFrame by 'eval_loss' inside the 'Evaluation Results' column
df_sorted = df.sort_values(by='Evaluation Results', 
                           key=lambda x: x.apply(lambda d: d['eval_loss']))



In [54]:
df_sorted.head()

,Hyperparameters,Evaluation Results,Training Output,Cosine Similarity
7,"{'learning_rate': 1e-05, 'num_train_epochs': 2...","{'eval_loss': 3.747753620147705, 'eval_runtime...","(3, 2.542485992113749, {'train_runtime': 3.056...",0.202080
6,"{'learning_rate': 1e-05, 'num_train_epochs': 1...","{'eval_loss': 3.842315673828125, 'eval_runtime...","(3, 2.943731149037679, {'train_runtime': 3.178...",0.202080
8,"{'learning_rate': 0.0001, 'num_train_epochs': ...","{'eval_loss': 3.955519914627075, 'eval_runtime...","(3, 2.735529979070028, {'train_runtime': 3.097...",0.000816
5,"{'learning_rate': 1e-05, 'num_train_epochs': 5...","{'eval_loss': 3.987677574157715, 'eval_runtime...","(3, 3.4048125743865967, {'train_runtime': 3.02...",0.272767
4,"{'learning_rate': 1e-05, 'num_train_epochs': 1...","{'eval_loss': 4.159300804138184, 'eval_runtime...","(3, 3.904948075612386, {'train_runtime': 3.377...",0.142329


In [58]:
# Sort the DataFrame by 'Cosine Similarity' from largest to smallest
df_cos = df.sort_values(by='Cosine Similarity', ascending=False)

In [59]:
df_cos

,Hyperparameters,Evaluation Results,Training Output,Cosine Similarity
5,"{'learning_rate': 1e-05, 'num_train_epochs': 5...","{'eval_loss': 3.987677574157715, 'eval_runtime...","(3, 3.4048125743865967, {'train_runtime': 3.02...",0.272767
3,"{'learning_rate': 1e-06, 'num_train_epochs': 2...","{'eval_loss': 4.3534440994262695, 'eval_runtim...","(3, 3.999807039896647, {'train_runtime': 2.944...",0.225828
0,"{'learning_rate': 1e-06, 'num_train_epochs': 1...","{'eval_loss': 4.424636363983154, 'eval_runtime...","(3, 4.1867062250773115, {'train_runtime': 3.24...",0.224779
1,"{'learning_rate': 1e-06, 'num_train_epochs': 5...","{'eval_loss': 4.399848461151123, 'eval_runtime...","(3, 4.1223320960998535, {'train_runtime': 2.91...",0.224779
2,"{'learning_rate': 1e-06, 'num_train_epochs': 1...","{'eval_loss': 4.3765645027160645, 'eval_runtim...","(3, 4.059745152791341, {'train_runtime': 2.928...",0.224779
6,"{'learning_rate': 1e-05, 'num_train_epochs': 1...","{'eval_loss': 3.842315673828125, 'eval_runtime...","(3, 2.943731149037679, {'train_runtime': 3.178...",0.202080
7,"{'learning_rate': 1e-05, 'num_train_epochs': 2...","{'eval_loss': 3.747753620147705, 'eval_runtime...","(3, 2.542485992113749, {'train_runtime': 3.056...",0.202080
4,"{'learning_rate': 1e-05, 'num_train_epochs': 1...","{'eval_loss': 4.159300804138184, 'eval_runtime...","(3, 3.904948075612386, {'train_runtime': 3.377...",0.142329
9,"{'learning_rate': 0.0001, 'num_train_epochs': ...","{'eval_loss': 4.3177995681762695, 'eval_runtim...","(3, 1.7846872011820476, {'train_runtime': 3.14...",0.003605
11,"{'learning_rate': 0.0001, 'num_train_epochs': ...","{'eval_loss': 5.07048225402832, 'eval_runtime'...","(3, 1.360078861316045, {'train_runtime': 3.104...",0.001679
